# Proyecto NLP: Clasificación de Noticias Reales vs Falsas

**Objetivo:** Entrenar un modelo capaz de predecir si un titular es real o falso, aplicando técnicas de preprocesamiento, vectorización, clasificación y análisis de sentimientos.

## 1. Carga de datos

In [4]:
## LIBRERIAS NECESARIAS PARA EL PROYECTO

# libreria necesaria para la manipulacion de datos

import pandas as pd

#Librerias necesarias para el procesamiento de texto

import nltk
import re
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Vectorizacion del texto y seleccion del modelo de clasificacion

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# analisis de sentimientos

from nltk.sentiment import SentimentIntensityAnalyzer

# prediccion del conjunto de prueba y exportacion de resultados

import joblib



In [5]:
# Leer los archivos CSV con separador tabulador
train = pd.read_csv('training_data.csv', sep='\t', header=None, names=['label', 'title'])
test = pd.read_csv('testing_data.csv', sep='\t', header=None, names=['label', 'title'])

print(train.head())
print(test.head())


   label                                              title
0      0  donald trump sends out embarrassing new year‚s...
1      0  drunk bragging trump staffer started russian c...
2      0  sheriff david clarke becomes an internet joke ...
3      0  trump is so obsessed he even has obama‚s name ...
4      0  pope francis just called out donald trump duri...
  label                                              title
0     2  copycat muslim terrorist arrested with assault...
1     2  wow! chicago protester caught on camera admits...
2     2   germany's fdp look to fill schaeuble's big shoes
3     2  mi school sends welcome back packet warning ki...
4     2  u.n. seeks 'massive' aid boost amid rohingya '...


In [6]:
train.head(5)  ## muestra las primeras 5 filas del dataframe train

,label,title
0,0,donald trump sends out embarrassing new year‚s...
1,0,drunk bragging trump staffer started russian c...
2,0,sheriff david clarke becomes an internet joke ...
3,0,trump is so obsessed he even has obama‚s name ...
4,0,pope francis just called out donald trump duri...


In [7]:
train.shape  # muestra la forma del dataframe train

(34152, 2)

In [8]:
print(train.columns) # muestra las columnas del dataframe train

Index(['label', 'title'], dtype='object')


In [9]:
test.head(5) # muestra las primeras 5 filas del dataframe test

,label,title
0,2,copycat muslim terrorist arrested with assault...
1,2,wow! chicago protester caught on camera admits...
2,2,germany's fdp look to fill schaeuble's big shoes
3,2,mi school sends welcome back packet warning ki...
4,2,u.n. seeks 'massive' aid boost amid rohingya '...


In [10]:
print(test.columns) # muestra las columnas del dataframe test

Index(['label', 'title'], dtype='object')


In [11]:

# Vista rápida de los datos y su distribución de clases asi como la forma de los dataframes y columnas
print("Training Data (primeras 5 filas):")
print(train.head())

print("\nDistribución de clases en training_data:")
print(train['label'].value_counts())

print("\nTesting Data (primeras 5 filas):")
print(test.head())

print("\nEtiquetas en testing_data:")
print(test['label'].value_counts())


Training Data (primeras 5 filas):
   label                                              title
0      0  donald trump sends out embarrassing new year‚s...
1      0  drunk bragging trump staffer started russian c...
2      0  sheriff david clarke becomes an internet joke ...
3      0  trump is so obsessed he even has obama‚s name ...
4      0  pope francis just called out donald trump duri...

Distribución de clases en training_data:
label
0    17572
1    16580
Name: count, dtype: int64

Testing Data (primeras 5 filas):
  label                                              title
0     2  copycat muslim terrorist arrested with assault...
1     2  wow! chicago protester caught on camera admits...
2     2   germany's fdp look to fill schaeuble's big shoes
3     2  mi school sends welcome back packet warning ki...
4     2  u.n. seeks 'massive' aid boost amid rohingya '...

Etiquetas en testing_data:
label
2     9982
﻿0       2
Name: count, dtype: int64


## 2. Preprocesamiento de texto

In [12]:
# Descargar recursos de NLTK necesarios para el procesamiento de texto

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Cargar los datos usando separador tabulador '\t'
train = pd.read_csv('training_data.csv', sep='\t', header=None, names=['label', 'title'])
test = pd.read_csv('testing_data.csv', sep='\t', header=None, names=['label', 'title'])

# Inicializar el lematizador y las stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) - {'no', 'not'}  # Retener negaciones

# Función de preprocesamiento mejorada
def preprocess_text(text):
    text = text.lower()  # Minúsculas
    text = re.sub(r'http\S+|www\S+', '', text)  # Quitar URLs
    text = re.sub(r'\d+', '', text)  # Quitar números
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)  # Quitar puntuación
    text = re.sub(r'\s+', ' ', text).strip()  # Quitar espacios extra

    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)

# Aplicar el preprocesamiento
train['clean_text'] = train['title'].apply(preprocess_text)
test['clean_text'] = test['title'].apply(preprocess_text)

# Mostrar ejemplos de datos procesados
print("Ejemplos limpios:")
print(train[['label', 'title', 'clean_text']].head())




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luisc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luisc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luisc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Ejemplos limpios:
   label                                              title  \
0      0  donald trump sends out embarrassing new year‚s...   
1      0  drunk bragging trump staffer started russian c...   
2      0  sheriff david clarke becomes an internet joke ...   
3      0  trump is so obsessed he even has obama‚s name ...   
4      0  pope francis just called out donald trump duri...   

                                          clean_text  
0  donald trump sends embarrassing new year‚s eve...  
1  drunk bragging trump staffer started russian c...  
2  sheriff david clarke becomes internet joke thr...  
3  trump obsessed even obama‚s name coded website...  
4  pope francis called donald trump christmas speech  


## 3. Vectorización del texto y selección de modelo

In [13]:


# Vectorización con TF-IDF
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Unigrama y bigrama
X = tfidf.fit_transform(train['clean_text'])
y = train['label']
X_test_final = tfidf.transform(test['clean_text'])

# Dividir el conjunto de entrenamiento para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar modelos
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Entrenamiento y evaluación de modelos
print("🔍 Comparando modelos...\n")
best_model = None
best_score = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    acc = accuracy_score(y_val, predictions)
    print(f"📊 {name} Accuracy (en validación): {acc:.4f}")

    # Evaluación con validación cruzada
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f"📊 {name} Accuracy media (CV): {scores.mean():.4f} ± {scores.std():.4f}")
    
    if acc > best_score:
        best_score = acc
        best_model = model

print("\n✅ Mejor modelo (por validación):", best_model.__class__.__name__)



🔍 Comparando modelos...

📊 Logistic Regression Accuracy (en validación): 0.9334
📊 Logistic Regression Accuracy media (CV): 0.9025 ± 0.0246
📊 Naive Bayes Accuracy (en validación): 0.9265
📊 Naive Bayes Accuracy media (CV): 0.9034 ± 0.0220
📊 Random Forest Accuracy (en validación): 0.9145
📊 Random Forest Accuracy media (CV): 0.8721 ± 0.0379

✅ Mejor modelo (por validación): LogisticRegression


## 4. Entrenamiento del modelo final

In [14]:

# Entrenar el mejor modelo (en este caso, Logistic Regression)
final_model = LogisticRegression(max_iter=200)
final_model.fit(X, y)


LogisticRegression(max_iter=200)

## 5. Análisis de sentimientos

In [15]:
# Guardar el modelo entrenado

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    return sia.polarity_scores(text)['compound']

train['sentiment'] = train['title'].apply(get_sentiment)
test['sentiment'] = test['title'].apply(get_sentiment)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luisc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## 6. Predicción en el conjunto de prueba y exportación

In [16]:



# Realizar predicciones
preds = final_model.predict(X_test_final)
preds_proba = final_model.predict_proba(X_test_final)[:, 1]

# Crear archivo de salida con las predicciones
test_output = test.copy()
test_output['label'] = preds
test_output[['label', 'title']].to_csv('predicciones_testing_data.csv', sep='\t', index=False, header=False)

# Guardar modelo y vectorizador
joblib.dump(final_model, 'modelo_noticias.pkl')
joblib.dump(tfidf, 'vectorizador_tfidf.pkl')

print("Archivo de predicciones y modelos guardados correctamente.")


Archivo de predicciones y modelos guardados correctamente.


## 7. Clasificación manual de un nuevo titular (con análisis de sentimiento)

In [19]:
# Clasificación manual de un nuevo titular (con análisis de sentimiento)


# cargar el modelo y el vectorizador
best_model = joblib.load('modelo_noticias.pkl')
tfidf = joblib.load('vectorizador_tfidf.pkl')

# Inicializar el analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Función para limpiar texto
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower()

# Ingresar nuevo titular
input_title = input("✏️ Ingresa un titular de noticia: ")

# Limpiar y vectorizar el titular
vect_title = tfidf.transform([cleaned])  # `cleaned` is already defined in the notebook

# Predecir si es real o falsa
prediction = best_model.predict(vect_title)[0]
probability = best_model.predict_proba(vect_title)[0]

# Analizar sentimiento
sentiment = sia.polarity_scores(cleaned)['compound']

# Mostrar resultados
print("\n📌 Resultado del análisis:")
print("✅ La noticia parece REAL" if prediction == 1 else "❌ La noticia parece FALSA")
print(f"🔢 Probabilidad de ser REAL: {probability[1]:.2f}")
print(f"💬 Sentimiento (compound): {sentiment:.2f}")
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower()

# Ingresar nuevo titular
input_title = input("✏️ Ingresa un titular de noticia: ")

# Limpiar y vectorizar el titular
cleaned = clean_text(input_title)
vect_title = tfidf.transform([cleaned])

# Predecir si es real o falsa
prediction = best_model.predict(vect_title)[0]
probability = best_model.predict_proba(vect_title)[0]

# Analizar sentimiento
sentiment = sia.polarity_scores(cleaned)['compound']

# Mostrar resultados
print("\n📌 Resultado del análisis:")
print("✅ La noticia parece REAL" if prediction == 1 else "❌ La noticia parece FALSA")
print(f"🔢 Probabilidad de ser REAL: {probability[1]:.2f}")
print(f"💬 Sentimiento (compound): {sentiment:.2f}")


📌 Resultado del análisis:
✅ La noticia parece REAL
🔢 Probabilidad de ser REAL: 0.51
💬 Sentimiento (compound): 0.00

📌 Resultado del análisis:
✅ La noticia parece REAL
🔢 Probabilidad de ser REAL: 0.51
💬 Sentimiento (compound): 0.00
